# cobrascape: 
### constraint-based reconstruction and analysis (COBRA) for microbial fitness landscapes

Load input data
- `Genetic variant matrix`
- `Genome-scale metabolic model`

In [1]:
from cobra.io import load_json_model
import pandas as pd

In [2]:
DATA_DIR = "example_data/"
MODEL_FILE = DATA_DIR + 'iEK1011_drugTesting_media.json'
X_ALLELES_FILE = DATA_DIR + 'X_ALLELES_FILE.csv'  

X_strains_alleles = pd.read_csv(X_ALLELES_FILE, index_col = 0)
# only look at first 20 strains for example
X_strains_alleles = X_strains_alleles.iloc[:20]
print("input: (G)enetic variant matrix= (strains: %d, alleles: %d)" % (X_strains_alleles.shape[0], X_strains_alleles.shape[1]))

COBRA_MODEL = load_json_model(MODEL_FILE)
print("input: (S)toichimetric genome-scale model= (genes: %d, reactions: %d, metabolites: %d)" % (len(COBRA_MODEL.genes), 
    len(COBRA_MODEL.reactions), len(COBRA_MODEL.metabolites)))

input: (G)enetic variant matrix= (strains: 20, alleles: 12762)
input: (S)toichimetric genome-scale model= (genes: 1011, reactions: 1229, metabolites: 998)


Create cobrascape object of data

In [3]:
import cobrascape.species as cs

In [4]:
### Create Species object
SPECIES_MODEL = cs.Species("species_obj")
COBRA_MODEL.solver = "glpk"
SPECIES_MODEL.base_cobra_model = COBRA_MODEL
SPECIES_MODEL.load_from_matrix(X_strains_alleles, filter_model_genes=True, allele_gene_sep="_")
for allele in SPECIES_MODEL.alleles:
    allele.cobra_gene = allele.id.split("_")[0]
SPECIES_MODEL.update_strains_cobra_model() # This creates a unique GEM for each strain

# alleles: 12762 -> removing alleles not in GEM -> # alleles after: 3310


100%|██████████| 20/20 [00:04<00:00,  4.82it/s]


There are 3310 alleles in the data that can be modeled with enzymatic flux in COBRA.

In [5]:
SPECIES_MODEL

Name,species_obj
Memory address,0x013214316a0
Number of alleles,3310
Number of strains,20
base cobra model,iEK1011


In [7]:
# Look at alleles of the gene Rv2524c and the strains they are in
for x in SPECIES_MODEL.alleles[21:28]:
    print(x, x.strains)

Rv2524c_1 set()
Rv2524c_2 set()
Rv2524c_3 set()
Rv2524c_4 {<Strain 1262526_3 at 0x1321421f28>, <Strain 1245787_3 at 0x1321421940>, <Strain 1262525_3 at 0x1321421f60>, <Strain 1138877_3 at 0x132142b978>}
Rv2524c_5 {<Strain 1126682_4 at 0x132142b208>, <Strain 1160714_3 at 0x132142ba20>, <Strain 1249615_4 at 0x1321421be0>, <Strain 1010835_3 at 0x1321431c50>, <Strain 1151113_3 at 0x132142b940>, <Strain 1078763_3 at 0x132142b160>, <Strain 1010834_3 at 0x1321431b70>, <Strain 1160716_3 at 0x132142bf60>, <Strain 1160715_3 at 0x132142bbe0>, <Strain 1240677_3 at 0x13214217f0>}
Rv2524c_6 set()
Rv2524c_7 set()


In [8]:
# Which strains contain allele Rv2524c_4?
SPECIES_MODEL.alleles.get_by_id("Rv2524c_4").strains

{<Strain 1138877_3 at 0x132142b978>,
 <Strain 1245787_3 at 0x1321421940>,
 <Strain 1262525_3 at 0x1321421f60>,
 <Strain 1262526_3 at 0x1321421f28>}

In [9]:
# What strain-specific GEM reactions does allele Rv2524c_4 encode?
SPECIES_MODEL.alleles.get_by_id("Rv2524c_4").cobra_reactions

{'FAS240_L': [<Reaction FAS240_L at 0x1323311828>,
  <Reaction FAS240_L at 0x13311c7518>,
  <Reaction FAS240_L at 0x1331f73470>,
  <Reaction FAS240_L at 0x1332632c50>],
 'FAS100': [<Reaction FAS100 at 0x1323311668>,
  <Reaction FAS100 at 0x13311c7358>,
  <Reaction FAS100 at 0x1331f732b0>,
  <Reaction FAS100 at 0x1332632a90>],
 'FAS260': [<Reaction FAS260 at 0x1323311860>,
  <Reaction FAS260 at 0x13311c7550>,
  <Reaction FAS260 at 0x1331f734a8>,
  <Reaction FAS260 at 0x1332632c88>],
 'FAS80_L': [<Reaction FAS80_L at 0x1323311898>,
  <Reaction FAS80_L at 0x13311c7588>,
  <Reaction FAS80_L at 0x1331f734e0>,
  <Reaction FAS80_L at 0x1332632cc0>],
 'FAS120': [<Reaction FAS120 at 0x13233116a0>,
  <Reaction FAS120 at 0x13311c7390>,
  <Reaction FAS120 at 0x1331f732e8>,
  <Reaction FAS120 at 0x1332632ac8>],
 'FAS140': [<Reaction FAS140 at 0x13233116d8>,
  <Reaction FAS140 at 0x13311c73c8>,
  <Reaction FAS140 at 0x1331f73320>,
  <Reaction FAS140 at 0x1332632b00>],
 'FAS160': [<Reaction FAS160 at

### Compute population-level FBA and FVA for the 200 strains

In [10]:
# FBA
%time SPECIES_MODEL.optimize_strains()
SPECIES_MODEL.solution

CPU times: user 52 ms, sys: 41.9 ms, total: 93.9 ms
Wall time: 521 ms


[('1010834_3', <Solution 0.058 at 0x1332bfb400>),
 ('1010835_3', <Solution 0.058 at 0x1332c0f5c0>),
 ('1010836_3', <Solution 0.058 at 0x1332c0f438>),
 ('1078763_3', <Solution 0.058 at 0x1332bfb5c0>),
 ('1126682_4', <Solution 0.058 at 0x1332c1def0>),
 ('1126683_4', <Solution 0.058 at 0x1332c32d30>),
 ('1126684_4', <Solution 0.058 at 0x1332c32da0>),
 ('1138877_3', <Solution 0.058 at 0x1332c8c4a8>),
 ('1151113_3', <Solution 0.058 at 0x1332c8c518>),
 ('1160714_3', <Solution 0.058 at 0x1332cad8d0>),
 ('1160715_3', <Solution 0.058 at 0x1332cceba8>),
 ('1160716_3', <Solution 0.058 at 0x1332d31710>),
 ('1200347_3', <Solution 0.058 at 0x1332d10320>),
 ('1200348_3', <Solution 0.058 at 0x1332d10390>),
 ('1240677_3', <Solution 0.058 at 0x1332d519e8>),
 ('1245275_3', <Solution 0.058 at 0x1332d74da0>),
 ('1245787_3', <Solution 0.058 at 0x1332d97160>),
 ('1249615_4', <Solution 0.058 at 0x1332db84e0>),
 ('1262525_3', <Solution 0.058 at 0x1332dd8860>),
 ('1262526_3', <Solution 0.058 at 0x1332df9be0>)]

Now try without parallelization...

In [11]:
%time SPECIES_MODEL.optimize_strains(parallel=False)
SPECIES_MODEL.solution

100%|██████████| 20/20 [00:01<00:00, 15.17it/s]

CPU times: user 1.34 s, sys: 21.6 ms, total: 1.36 s
Wall time: 1.36 s


{'1010834_3': <Solution 0.058 at 0x1332be8828>,
 '1010835_3': <Solution 0.058 at 0x1332be84a8>,
 '1010836_3': <Solution 0.058 at 0x1332be8eb8>,
 '1078763_3': <Solution 0.058 at 0x1332e7fa58>,
 '1126682_4': <Solution 0.058 at 0x1332bfb2e8>,
 '1126683_4': <Solution 0.058 at 0x1332eb04e0>,
 '1126684_4': <Solution 0.058 at 0x1332eb0630>,
 '1138877_3': <Solution 0.058 at 0x1332eb07f0>,
 '1151113_3': <Solution 0.058 at 0x1332eb0940>,
 '1160714_3': <Solution 0.058 at 0x1332eb0b00>,
 '1160715_3': <Solution 0.058 at 0x1332eb0c50>,
 '1160716_3': <Solution 0.058 at 0x1332eb0e10>,
 '1200347_3': <Solution 0.058 at 0x1332eb0f60>,
 '1200348_3': <Solution 0.058 at 0x1332bc1cf8>,
 '1240677_3': <Solution 0.058 at 0x1332ebf128>,
 '1245275_3': <Solution 0.058 at 0x1332ebf2b0>,
 '1245787_3': <Solution 0.058 at 0x1332ebf470>,
 '1249615_4': <Solution 0.058 at 0x1332ebf5c0>,
 '1262525_3': <Solution 0.058 at 0x1332ebf710>,
 '1262526_3': <Solution 0.058 at 0x1332ebf8d0>}

Parallelization is significantly faster. This becomes more apparent with larger number of strains and while running popFVA

In [12]:
# FVA
%time SPECIES_MODEL.optimize_strains(fva=True)

CPU times: user 54.1 ms, sys: 43.9 ms, total: 98 ms
Wall time: 10.4 s


### NOTE: We see no difference in strain solutions above because no allele-specific parameters have been set!

To compute an allele-parameterized population, provide a mapping of allele to flux bound and then use `cs.compute_constrained_species()`

### see _Metabolic Network Classifiers_ github for more details